In [1]:
import sys

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

sys.path.append("../RD-systems-and-test-benches/utils")
import model_fsolve as modf
import model_fsolve_v2 as modf2

import scipy.optimize as sco
import scipy.interpolate as sci

import hx_hydraulic as hxhy
import from_excel as fe
import data_plots as dplt

import fluids as fds
from CoolProp.CoolProp import PropsSI

import openpyxl as opxl
from sklearn.ensemble import RandomForestRegressor

import seaborn as sns
import os
import plotly.graph_objects as go

import utils.data_processing as dp
import utils.conversion as conv
import copy

from plot_config import *

In [2]:
path = r"G:\Drive partagés\Cercle Hard\R&D\Modèles numériques PVT\PVT-PL-model\Inputs"
file_name = "\V4.5_wo_interMPE.xlsx"
hw, par, cond = fe.initialize(path, file_name)

fluid = {'name':'MPG','glycol_rate':0.4}

# tabl, res, PL, residuals = modf.PL_fsolve(par, cond) 

In [9]:
df, PL, df_PL, residuals = modf.PL_fsolve(par, cond) 

In [ ]:
PL

In [ ]:
plt.scatter(df.index, df['qx']/(df['qx'].sum()/304), label='qx')

### Series for multiple total flow rates (Qdot)

In [ ]:
Vdot_list = [600]
# list_Qtot = np.arange(50,1050, 50)/3600000
Qtot_list = [conv.convert_flow_rate(Vdot, 'L/h', 'm3/s') for Vdot in Vdot_list]

N = 6

testings_path_list = [os.path.join(r"G:\Drive partagés\Cercle Hard\R&D\Modèles numériques PVT\PVT-PL-model\Thesis", file_name[1:-5] + f"_test_{int(Vdot)}Lh.xlsx") for Vdot in Vdot_list]

for i, Qtot in enumerate(Qtot_list):
    # list_Qin = np.arange(0.1, 1.1, 0.1)*Qtot
    # list_alpha = np.arange(0.05, 1.05, 0.05)
    list_Qin = np.linspace(0.05, 0.95, 7)*Qtot
    list_alpha = np.linspace(0.05, 0.95, 7)

    df_testings = fe.testing_series_Qmax(Qtot, list_Qin, list_alpha, par,cond)
    testings_path = testings_path_list[i]
    df_testings.to_excel(testings_path, index=False)


Vdot = Vdot_list[0]
testings_path = testings_path_list[0]

df_testings = pd.read_excel(testings_path)
DPin, DPout, DPx = modf2.transfer_func(df_testings)
tabl, residuals = modf2.PL_fsolve_MPE(N, conv.convert_flow_rate(Vdot, 'L/h', 'm3/s'), DPin, DPout, DPx)

### Series for one flow rate but changing geometry

In [4]:
def find_other_dimension(Dh, side):
    """
    Calculate the height h of a rectangular tube given the hydraulic diameter Dh and width w.
    """
    if 2 * side <= Dh:
        raise ValueError("Invalid dimensions: 2 * w should be >  Dh")
    return (Dh * side) / (2 * side - Dh)

In [5]:
D_riser_range = np.array([1.4,1.2,1.,0.8,0.6]) * par['D_riser']

In [ ]:
hw, par, cond = fe.initialize(path, file_name)
N_panels = 6

Vdot = 600
Qtot = conv.convert_flow_rate(Vdot, 'L/h', 'm3/s')

h_riser_range = [find_other_dimension(D_riser, par['w_riser']) for D_riser in D_riser_range]
print(h_riser_range)
res_dict_h = {}

testings_path_list = [os.path.join(r"G:\Drive partagés\Cercle Hard\R&D\Modèles numériques PVT\PVT-PL-model\Thesis", file_name[1:-5] + f"change_h_riser_{int(h_riser*1000)}.xlsx") for h_riser in h_riser_range]

for i, h_riser in enumerate(h_riser_range):
    print('i ', i)

    # Change diameter

    hw.change_riser_height(h_riser)
    par = hw.make_dict()

    # Generate abaque

    list_Qin = np.linspace(0.05, 0.95, 7)*Qtot
    list_alpha = np.linspace(0.05, 0.95, 7)

    df_testings = fe.testing_series_Qmax(Qtot, list_Qin, list_alpha, par,cond)
    testings_path = testings_path_list[i]
    df_testings.to_excel(testings_path, index=False)

    # Run abaque for flow rate Vdot (L/h)

    testings_path = testings_path_list[i]

    df_testings = pd.read_excel(testings_path)
    DPin, DPout, DPx = modf2.transfer_func(df_testings)
    tabl, residuals = modf2.PL_fsolve_MPE(N_panels , conv.convert_flow_rate(Vdot, 'L/h', 'm3/s'), DPin, DPout, DPx)

    res_dict_h[h_riser] = {
                        'N_panels' : N_panels,
                        'Vdot' : Vdot,
                        'tabl' : copy.deepcopy(tabl),
                        'residuals' : copy.deepcopy(residuals)
                        }

    print('---')

In [ ]:
hw, par, cond = fe.initialize(path, file_name)
N_panels = 6

Vdot = 600
Qtot = conv.convert_flow_rate(Vdot, 'L/h', 'm3/s')

w_riser_range = [find_other_dimension(D_riser, par['h_riser']) for D_riser in D_riser_range]
print(w_riser_range)
res_dict_w = {}

testings_path_list = [os.path.join(r"G:\Drive partagés\Cercle Hard\R&D\Modèles numériques PVT\PVT-PL-model\Thesis", file_name[1:-5] + f"change_h_riser_{int(w_riser*1000)}.xlsx") for w_riser in w_riser_range]

for i, w_riser in enumerate(w_riser_range):
    print('i ', i)

    # Change diameter

    hw.change_riser_width_and_update_N(w_riser)
    par = hw.make_dict()

    # Generate abaque

    list_Qin = np.linspace(0.05, 0.95, 7)*Qtot
    list_alpha = np.linspace(0.05, 0.95, 7)

    df_testings = fe.testing_series_Qmax(Qtot, list_Qin, list_alpha, par,cond)
    testings_path = testings_path_list[i]
    df_testings.to_excel(testings_path, index=False)

    # Run abaque for flow rate Vdot (L/h)

    testings_path = testings_path_list[i]

    df_testings = pd.read_excel(testings_path)
    DPin, DPout, DPx = modf2.transfer_func(df_testings)
    tabl, residuals = modf2.PL_fsolve_MPE(N_panels , conv.convert_flow_rate(Vdot, 'L/h', 'm3/s'), DPin, DPout, DPx)

    res_dict_w[w_riser] = {
                        'N_panels' : N_panels,
                        'Vdot' : Vdot,
                        'tabl' : copy.deepcopy(tabl),
                        'residuals' : copy.deepcopy(residuals)
                        }

    print('---')

In [38]:
hw, par, cond = fe.initialize(path, file_name)

for (letter, res_dict, x_range) in zip(['h', 'w'], [res_dict_h, res_dict_w], [h_riser_range, w_riser_range]):

    res_dict['parameters'] = {
                                'par' : copy.deepcopy(par),
                                'cond' : copy.deepcopy(cond),
                                'N_panels' : N_panels,
                                'Vdot' : Vdot,
                                'Qtot' : Qtot,
                                'x_range' : x_range
                                }

    dp.write_pickle(r"G:\Drive partagés\Cercle Hard\R&D\Modèles numériques PVT\PVT-PL-model\Thesis", f"V4.5_multipanels_parametric_res_dict_{letter}", res_dict)

### Start again here

In [ ]:
for key in res_dict_h.keys():
    # key = list(res_dict.keys())[1]
    tabl = res_dict_h[key]['tabl']
    plt.scatter(tabl.index, tabl['qx']/(tabl['qx'].sum()/N_panels), label=f'h_riser = {key*1000} mm')
    print(tabl['Pin'][0])

plt.legend()

In [ ]:
for key in res_dict_w.keys():
    # key = list(res_dict.keys())[1]
    tabl = res_dict_w[key]['tabl']
    plt.scatter(tabl.index, tabl['qx']/(tabl['qx'].sum()/N_panels), label=f'w_riser = {key*1000} mm')
    print(tabl['Pin'][0])

plt.legend()